<a href="https://colab.research.google.com/github/ktichola/Image-matching/blob/main/%F0%9F%8C%9F%F0%9F%9A%80Hexathlon_IMC%7B_DinoV2_%7D%F0%9F%9A%80%F0%9F%8C%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'image-matching-challenge-2024:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F71885%2F8143495%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T112612Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D59f7f3eaf4da62c477575a3a352f0c5f5ae59817aebfa62345b2c79888b57a572abde5452ea710174fc9c0d06deaf1efc4ac5f89bb2f1c31b509d16a0ea5580493dd11ad70191ca10fd02dbe4e68f733dcc522c8c04be3151462e08af2d2368f9778be75afc69e07eff2158425a324be54798637a2598801cda6bf9228e3c5f85dce4bd5c9d01c770b964bb77e1b36655ff5127607f00b24d9b1d62ccdf25f10e0bba7bcdd46e1c0226fea3330ba8c04d4d5aabf215139343c17b4b48af72c6cb0d9a3dc44a8ab0fee738d079950ab2e9924fd41dea41fd096b8e574691fcbf0b1b1e777bdffefb6b30b1af81c62498e42cd7f17d2e605e6c3b10e9507286dec,imc2024-packages-lightglue-rerun-kornia:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628051%2F7884485%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T112612Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D57111324cb3ca7f9b3e96275fe36912b1288a300af2db325cf765084b9465eb61af2dca2aabf0aaa7f205844017488a0a6c33e1a25736ae5ea8e1453ed7fbe109f52f32c43507f90eaff6e57b03ed34dd841d0777d485d8043ffc362f8bfd189221840e6461cf66c00ce6cc9e41d7b852f4eb61b027801f8089ded69596d6af4340704b34f6b8c2f644af842a15b93dd1606ece36020f475c430625baf0001b957e4e6f7aaabb85a58c980ec5882585413c966c9f5b1e1e6eba93ee5abbae5c53f1778de20760f5cbd2b232a7b5f806cca73834b8873f82369de6438d09941de6f8838f15213535f28b08035849f698e275e94ca7f460092d67596d1d316a18c,colmap-db-import:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4628331%2F7884725%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T112612Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D41b8bd7ebca920eec37bffa54b751c0d7ae410241c96c310af565b49f9bc735237c6dece67a04eaeece3e312c5ffb72a6df70e8f819c304721dc386674ddfc073d18f43e9f131895b782b25a6eefcd49ad7e16cd143202c7614ebe04aca77857a858513b2ea6136f50667cd743315fb9ebfb15cf94dd19a289a48fd57d4014d8b5f3a6410c6673aac51e00fa275563dba173b503e9b805eaf4637442ef66d7beecd948c084ccb0ed5b0d6e2f4ea472c546ebc5b12064acb72b0ddd05368c51f432964931ed19542080b1bf8d00fd51a84e88ee0d1047f6c71f03656346adfd29845f182caebe452691813d20606438240da2e6cd4b50e468fff2dbfd527ba797,dinov2/pytorch/base/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F3326%2F4534%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T112612Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D61d34a9ded914084edadf06831a58a71645a6f1b82d3cdf481e0d4c7ea5788b574cb6060d095c70788771d98e822714886a7c2239055d0062efb6617a57391d76c4700cfdd649af9aa8ae5b7aa42749b458e89795f10d208f9dc0ae0b46b200dad222799611fd0b42b7c4c76582dfe7b9777c8827e1f39b73406988e272ed561b482e34a82299e9bdb9782b046cd23c23e985cf354a64257aedc62bc1eaeeaf54b588aaf816d13bc372a79de64dc1a1fb83bb864d961e6a00096476dfc46147f182939c9a2052e37176013d738fd3a5c6b5f4bfb0fb29ce12ebae78b37377f66a10236338b04342d84f24511e2098332d98782a9b748dc3c2b0be18c29457cd1,lightglue/pytorch/aliked/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14317%2F17191%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T112612Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D55a7421bcdc4f1bbdc1cfb32d85dec49ecc1f24ee391e7f26e66c9f8d0495228e36c4a7ef9dc287b72a7aa08cbbf7bfc0eeebf822a8a6f3f8f398fc087b7b07e64d6afecf0ffae887b4f4058079811ae10455c51956efa5ed09980c3f90ee58d0c6adf00929ebacd95b034844b902ac0a81a4d2e327a00765c1b77c9832466aaa0a643748a458e7f2d499073c542ffb1c6d85def07953d4294a05714b1128390e7d42b8233bfd3982d51f3bc8d49b17509a2181f7a290a882da1e224360090342703539943fbbd74489996ce078bcd931279449682af292e731321b431bfd28cdc3dbd9eef6966ff9cdb9b29f008901d0b41624d0287b26c42c423b20e698299,aliked/pytorch/aliked-n16/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F14611%2F17555%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240514%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240514T112612Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7bf6a75e2ac1f57af27bf367259dc2f9c312815c0fd2b640330d1e5ee388944ca703e8ce056893a9ae760bc7d74ca7e1d8d2e01f8b220855bc9eeca364828662d515f30df1726973e696d7319f1bc2e7c283e5d0c245ed7130fe9646bfc11b58d7056720dab7bb2f5f7b52262a2afffd407759f65b984045c3ddd3d6974ca3bb822c1683e70f6b3edb476e2aad4c4da9be025a37acca39f64fbea8ea17fb722eeb7f3b35572fea48a9153d092e41d3a478b56c116cbc0999decf5d54c64458dffa4454201d6a98dd6909713a2f6c6a39f0f036b974ec5fa77d7d8476a26cc52abee8090f4dd59f1f0186eadafc98fd45908d040af366e2dd1f1c3bb2a87e9bee'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# **Install userfull library:**

In [ ]:
!pip install --no-index /kaggle/input/imc2024-packages-lightglue-rerun-kornia/* --no-deps
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp /kaggle/input/aliked/pytorch/aliked-n16/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/* /root/.cache/torch/hub/checkpoints/
!cp /kaggle/input/lightglue/pytorch/aliked/1/aliked_lightglue.pth /root/.cache/torch/hub/checkpoints/aliked_lightglue_v0-1_arxiv-pth

Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia-0.7.2-py2.py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_moons-0.2.9-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/kornia_rs-0.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/lightglue-0.0-py3-none-any.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/pycolmap-0.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/imc2024-packages-lightglue-rerun-kornia/rerun_sdk-0.15.0a2-cp38-abi3-manylinux_2_31_x86_64.whl
kornia is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
  Attempting uninstall: kornia-rs
    Found existing installation: kornia_rs 0.1.3
    Uninstalling kornia_rs-0.1.3:
      Successfully uninstalled kornia_rs-0.1.3


In [ ]:
import matplotlib.pyplot as plt

from tqdm import tqdm
from time import time, sleep
from fastprogress import progress_bar
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
import matplotlib.pyplot as plt

import os
from tqdm import tqdm
from pathlib import Path
from time import time, sleep
from fastprogress import progress_bar
import gc
import numpy as np
import h5py
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from typing import Any
import itertools
import pandas as pd

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF
from PIL import Image
from transformers import AutoImageProcessor, AutoModel

import pycolmap

import sys
sys.path.append('/kaggle/input/colmap-db-import')
from database import *
from h5_to_db import *

def arr_to_str(a):
    return ';'.join([str(x) for x in a.reshape(-1)])

def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

device = K.utils.get_cuda_device_if_available(0)
print (device)

2024-05-14 11:19:01.173890: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 11:19:01.173993: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 11:19:01.316880: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


cuda:0


# **Building image pairs**

![image.png](attachment:ce6378f6-e22b-41ef-9749-0e2c453dd753.png)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Description of Code Functionality</title>
</head>
<body>

<h1>Description of Code Functionality</h1>

<h2>Function: get_global_desc</h2>
<p>The <code>get_global_desc</code> function extracts global descriptors from a set of images using a DINOv2 model. It preprocesses the images, passes them through the DINOv2 model, and extracts the global descriptors from the model's output. These descriptors are then normalized and returned.</p>

<h2>Function: get_img_pairs_exhaustive</h2>
<p>The <code>get_img_pairs_exhaustive</code> function generates all possible pairs of images from a list of image filenames. It iterates over all combinations of image indices, ensuring that each pair is unique.</p>

<h2>Function: get_image_pairs_shortlist</h2>
<p>The <code>get_image_pairs_shortlist</code> function generates pairs of images based on the similarity of their global descriptors. It calculates the pairwise distances between global descriptors, thresholding them based on a similarity threshold. If the number of images is below a certain threshold, it performs an exhaustive search for all possible pairs. Otherwise, it shortlists pairs based on similarity, ensuring a minimum number of pairs.</p>

<p>This code provides functions for generating pairs of images either exhaustively or based on global descriptor similarity, enabling downstream tasks such as image matching and registration.</p>

</body>
</html>


In [ ]:
def get_global_desc(fnames, device = torch.device('cpu')):
    processor = AutoImageProcessor.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = AutoModel.from_pretrained('/kaggle/input/dinov2/pytorch/base/1')
    model = model.eval()
    model = model.to(device)
    global_descs_dinov2 = []
    for i, img_fname_full in tqdm(enumerate(fnames),total= len(fnames)):
        key = os.path.splitext(os.path.basename(img_fname_full))[0]
        timg = load_torch_image(img_fname_full)
        with torch.inference_mode():
            inputs = processor(images=timg, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            dino_mac = F.normalize(outputs.last_hidden_state[:,1:].max(dim=1)[0], dim=1, p=2)
        global_descs_dinov2.append(dino_mac.detach().cpu())
    global_descs_dinov2 = torch.cat(global_descs_dinov2, dim=0)
    return global_descs_dinov2

def get_img_pairs_exhaustive(img_fnames):
    index_pairs = []
    for i in range(len(img_fnames)):
        for j in range(i+1, len(img_fnames)):
            index_pairs.append((i,j))
    return index_pairs

def get_image_pairs_shortlist(fnames,
                              sim_th = 0.6,
                              min_pairs = 20,
                              exhaustive_if_less = 20,
                              device=torch.device('cpu')):
    num_imgs = len(fnames)
    if num_imgs <= exhaustive_if_less:
        return get_img_pairs_exhaustive(fnames)
    descs = get_global_desc(fnames, device=device)
    dm = torch.cdist(descs, descs, p=2).detach().cpu().numpy()
    mask = dm <= sim_th
    total = 0
    matching_list = []
    ar = np.arange(num_imgs)
    already_there_set = []
    for st_idx in range(num_imgs-1):
        mask_idx = mask[st_idx]
        to_match = ar[mask_idx]
        if len(to_match) < min_pairs:
            to_match = np.argsort(dm[st_idx])[:min_pairs]
        for idx in to_match:
            if st_idx == idx:
                continue
            if dm[st_idx, idx] < 1000:
                matching_list.append(tuple(sorted((st_idx, idx.item()))))
                total+=1
    matching_list = sorted(list(set(matching_list)))
    return matching_list

# **Self-supervised Keypoint Learning — A Review**

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>Keypoint Detection in Computer Vision</title>
</head>
<body>

<h1>Keypoint Detection in Computer Vision</h1>

<p>Keypoint or interest point detection is one important building block for many computer vision tasks, such as SLAM (simultaneous localization and mapping), SfM (structure from motion), and camera calibration. Keypoint detection has a long history predating deep learning, and many glorious algorithms in wide industry applications (such as FAST, SIFT, and ORB) are based on hand-crafted features. As in many other computer vision tasks, people have been exploring how to use deep learning to outperform hand-crafted algorithms. In this post, we will review some recent advances in this field.</p>

</body>
</html>

![image.png](attachment:8b1e36db-6628-4ad8-a031-67b52890f59b.png)

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title></title>
</head>
<body>

<h1>Description of Code Functionality</h1>

<p>This code snippet demonstrates a series of functions for keypoint detection, feature matching, and importing data into COLMAP, a Structure-from-Motion (SfM) and Multi-View Stereo (MVS) pipeline.</p>

<h2>Function: detect_aliked</h2>
<p>The <code>detect_aliked</code> function is responsible for detecting keypoints and extracting descriptors from images using the ALIKED model. Key points and descriptors are then stored in HDF5 files for each image.</p>

<h2>Function: match_with_lightglue</h2>
<p>The <code>match_with_lightglue</code> function matches features between pairs of images using the LightGlueMatcher model. It loads keypoints and descriptors from HDF5 files, performs feature matching, and saves the matches to another HDF5 file.</p>

<h2>Function: import_into_colmap</h2>
<p>The <code>import_into_colmap</code> function imports keypoints and matches into a COLMAP database. It connects to the database, adds keypoints and matches to it, and commits the changes.</p>

<p>This code serves as a pipeline for feature extraction, matching, and importing data into COLMAP for further 3D reconstruction and analysis.</p>

</body>
</html>


In [ ]:
import torch
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd

def detect_aliked(img_fnames,
                  feature_dir = '.featureout',
                  num_features = 4096,
                  resize_to = 1024,
                  device=torch.device('cpu')):
    dtype = torch.float32
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.01, resize=resize_to).eval().to(device, dtype)
    if not os.path.isdir(feature_dir):
        os.makedirs(feature_dir)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='w') as f_kp, \
         h5py.File(f'{feature_dir}/descriptors.h5', mode='w') as f_desc:
        for img_path in tqdm(img_fnames):
            img_fname = img_path.split('/')[-1]
            key = img_fname
            with torch.inference_mode():
                image0 = load_torch_image(img_path, device=device).to(dtype)
                feats0 = extractor.extract(image0)
                kpts = feats0['keypoints'].reshape(-1, 2).detach().cpu().numpy()
                descs = feats0['descriptors'].reshape(len(kpts), -1).detach().cpu().numpy()
                f_kp[key] = kpts
                f_desc[key] = descs
    return

def match_with_lightglue(img_fnames,
                   index_pairs,
                   feature_dir = '.featureout',
                   device=torch.device('cpu'),
                   min_matches=15,verbose=True):
    lg_matcher = KF.LightGlueMatcher("aliked", {"width_confidence": -1,
                                                "depth_confidence": -1,
                                                 "mp": True if 'cuda' in str(device) else False}).eval().to(device)
    with h5py.File(f'{feature_dir}/keypoints.h5', mode='r') as f_kp, \
        h5py.File(f'{feature_dir}/descriptors.h5', mode='r') as f_desc, \
        h5py.File(f'{feature_dir}/matches.h5', mode='w') as f_match:
        for pair_idx in tqdm(index_pairs):
            idx1, idx2 = pair_idx
            fname1, fname2 = img_fnames[idx1], img_fnames[idx2]
            key1, key2 = fname1.split('/')[-1], fname2.split('/')[-1]
            kp1 = torch.from_numpy(f_kp[key1][...]).to(device)
            kp2 = torch.from_numpy(f_kp[key2][...]).to(device)
            desc1 = torch.from_numpy(f_desc[key1][...]).to(device)
            desc2 = torch.from_numpy(f_desc[key2][...]).to(device)
            with torch.inference_mode():
                dists, idxs = lg_matcher(desc1,
                                         desc2,
                                         KF.laf_from_center_scale_ori(kp1[None]),
                                         KF.laf_from_center_scale_ori(kp2[None]))
            if len(idxs)  == 0:
                continue
            n_matches = len(idxs)
            if verbose:
                print (f'{key1}-{key2}: {n_matches} matches')
            group  = f_match.require_group(key1)
            if n_matches >= min_matches:
                 group.create_dataset(key2, data=idxs.detach().cpu().numpy().reshape(-1, 2))
    return

def import_into_colmap(img_dir,
                       feature_dir ='.featureout',
                       database_path = 'colmap.db'):
    db = COLMAPDatabase.connect(database_path)
    db.create_tables()
    single_camera = False
    fname_to_id = add_keypoints(db, feature_dir, img_dir, '', 'simple-pinhole', single_camera)
    add_matches(
        db,
        feature_dir,
        fname_to_id,
    )
    db.commit()
    return

In [ ]:
src = '/kaggle/input/image-matching-challenge-2024/'

data_dict = {}
with open(f'{src}/sample_submission.csv', 'r') as f:
    for i, l in enumerate(f):
        if i== 0:
            print (l)
        if l and i > 0:
            image_path, dataset, scene, _, _ = l.strip().split(',')
            if dataset not in data_dict:
                data_dict[dataset] = {}
            if scene not in data_dict[dataset]:
                data_dict[dataset][scene] = []
            data_dict[dataset][scene].append(image_path)
for dataset in data_dict:
    for scene in data_dict[dataset]:
        print(f'{dataset} / {scene} -> {len(data_dict[dataset][scene])} images')

out_results = {}
timings = {"shortlisting":[],
           "feature_detection": [],
           "feature_matching":[],
           "RANSAC": [],
           "Reconstruction": []}

image_path,dataset,scene,rotation_matrix,translation_vector

church / church -> 41 images


In [ ]:
def create_submission(out_results, data_dict):
    with open(f'submission.csv', 'w') as f:
        f.write('image_path,dataset,scene,rotation_matrix,translation_vector\n')
        for dataset in data_dict:
            if dataset in out_results:
                res = out_results[dataset]
            else:
                res = {}
            for scene in data_dict[dataset]:
                if scene in res:
                    scene_res = res[scene]
                else:
                    scene_res = {"R":{}, "t":{}}
                for image in data_dict[dataset][scene]:
                    if image in scene_res:
                        print (image)
                        R = scene_res[image]['R'].reshape(-1)
                        T = scene_res[image]['t'].reshape(-1)
                    else:
                        R = np.eye(3).reshape(-1)
                        T = np.zeros((3))
                    f.write(f'{image},{dataset},{scene},{arr_to_str(R)},{arr_to_str(T)}\n')

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">

</head>
<body>

<h1>Description of Code Functionality</h1>

<h2>Initialization and Data Preparation</h2>
<p>The code initializes variables and prepares the data for processing.</p>

<h2>Feature Extraction and Matching</h2>
<p>The code iterates over datasets and scenes, extracting features from images, shortlisting image pairs based on similarity, detecting keypoints and descriptors, matching features, and importing data into COLMAP for further processing.</p>

<h2>3D Reconstruction and Registration</h2>
<p>The code performs 3D reconstruction and registration tasks, including RANSAC matching, incremental mapping, selecting the best reconstruction, extracting rotation and translation matrices, and creating submission files.</p>

<h2>Error Handling</h2>
<p>Exception handling is implemented to catch any errors that may occur during the processing of datasets and scenes.</p>

<p>This code segment automates the process of feature extraction, matching, 3D reconstruction, and registration for multiple datasets and scenes, providing efficient data processing and feedback on the progress of each step.</p>

</body>
</html>


In [ ]:
gc.collect()
from copy import deepcopy
datasets = []
for dataset in data_dict:
    datasets.append(dataset)
print (f"Extracting on device {device}")
for dataset in data_dict:
    print(dataset)
    if dataset not in out_results:
        out_results[dataset] = {}
    for scene in data_dict[dataset]:
        print(scene)
        img_dir =  os.path.join(src,  '/'.join(data_dict[dataset][scene][0].split('/')[:-1]))
        try:
            out_results[dataset][scene] = {}
            img_fnames = [os.path.join(src, x) for x in data_dict[dataset][scene] ]
            print (f"Got {len(img_fnames)} images")
            feature_dir = f'featureout/{dataset}_{scene}'
            os.makedirs(feature_dir, exist_ok=True)
            t=time()
            index_pairs = get_image_pairs_shortlist(img_fnames,
                                  sim_th = 0.3,
                                  min_pairs = 20,
                                  exhaustive_if_less = 20,
                                  device=device)
            t=time() -t
            timings['shortlisting'].append(t)
            print (f'{len(index_pairs)}, pairs to match, {t:.4f} sec')
            gc.collect()
            t=time()
            detect_aliked(img_fnames, feature_dir, 4096, device=device)
            gc.collect()
            t=time() -t
            timings['feature_detection'].append(t)
            print(f'Features detected in  {t:.4f} sec')
            t=time()
            match_with_lightglue(img_fnames, index_pairs, feature_dir=feature_dir,device=device)
            t=time() -t
            timings['feature_matching'].append(t)
            print(f'Features matched in  {t:.4f} sec')
            database_path = f'{feature_dir}/colmap.db'
            if os.path.isfile(database_path):
                os.remove(database_path)
            gc.collect()
            sleep(1)
            import_into_colmap(img_dir, feature_dir=feature_dir, database_path=database_path)
            output_path = f'{feature_dir}/colmap_rec_aliked'
            t=time()
            pycolmap.match_exhaustive(database_path)
            t=time() - t
            timings['RANSAC'].append(t)
            print(f'RANSAC in  {t:.4f} sec')
            t=time()
            mapper_options = pycolmap.IncrementalPipelineOptions()
            mapper_options.min_model_size = 3
            mapper_options.max_num_models = 2
            os.makedirs(output_path, exist_ok=True)
            maps = pycolmap.incremental_mapping(database_path=database_path,
                                                image_path=img_dir,
                                                output_path=output_path, options=mapper_options)
            sleep(1)
            print(maps)
            clear_output(wait=False)
            t=time() - t
            timings['Reconstruction'].append(t)
            print(f'Reconstruction done in  {t:.4f} sec')
            imgs_registered  = 0
            best_idx = None
            print ("Looking for the best reconstruction")
            if isinstance(maps, dict):
                for idx1, rec in maps.items():
                    print (idx1, rec.summary())
                    try:
                        if len(rec.images) > imgs_registered:
                            imgs_registered = len(rec.images)
                            best_idx = idx1
                    except:
                        continue
            if best_idx is not None:
                print (maps[best_idx].summary())
                for k, im in maps[best_idx].images.items():
                    key1 = f'test/{scene}/images/{im.name}'
                    print(key1)
                    out_results[dataset][scene][key1] = {}
                    out_results[dataset][scene][key1]["R"] = deepcopy(im.cam_from_world.rotation.matrix())
                    out_results[dataset][scene][key1]["t"] = deepcopy(np.array(im.cam_from_world.translation))
            print(f'Registered: {dataset} / {scene} -> {len(out_results[dataset][scene])} images')
            print(f'Total: {dataset} / {scene} -> {len(data_dict[dataset][scene])} images')
            create_submission(out_results, data_dict)
            gc.collect()
        except Exception as e:
            print (e)
            pass

Reconstruction done in  154.6709 sec
Looking for the best reconstruction
0 Reconstruction:
	num_reg_images = 39
	num_cameras = 39
	num_points3D = 12288
	num_observations = 64013
	mean_track_length = 5.20939
	mean_observations_per_image = 1641.36
	mean_reprojection_error = 0.9047
Reconstruction:
	num_reg_images = 39
	num_cameras = 39
	num_points3D = 12288
	num_observations = 64013
	mean_track_length = 5.20939
	mean_observations_per_image = 1641.36
	mean_reprojection_error = 0.9047
test/church/images/00001.png
test/church/images/00006.png
test/church/images/00008.png
test/church/images/00010.png
test/church/images/00011.png
test/church/images/00012.png
test/church/images/00013.png
test/church/images/00018.png
test/church/images/00021.png
test/church/images/00024.png
test/church/images/00026.png
test/church/images/00029.png
test/church/images/00030.png
test/church/images/00032.png
test/church/images/00035.png
test/church/images/00037.png
test/church/images/00039.png
test/church/images/000

In [ ]:
!cat submission.csv

In [ ]:
Submission = pd.read_csv('submission.csv')

print(Submission)

In [ ]:
Submission.head()